In [ ]:
from win32com.client import Dispatch
import time
import numpy as np
from pyScope.operations.bin import pulsar_dimmer

In [ ]:
maxim = Dispatch("MaxIm.Application")
maxim.LockApp = True

In [ ]:
camera = Dispatch("MaxIm.CCDCamera")
camera.DisableAutoShutdown = True
camera.LinkEnabled = True
print('Camera connected: %s' % camera.CameraName); print()

readouts = camera.ReadoutModes
for i in range(len(readouts)):
    print('Mode %d: %s' % (i, readouts[i]))
print()
filter_names = camera.FilterNames
for i in range(len(filter_names)):
    print('Filter %d: %s' % (i, filter_names[i]))

In [ ]:
mount = Dispatch('SiTech.Telescope')
print('Homing')
mount.FindHome
mount.Tracking = False
print('Home found')

print('Slewing')
mount.SlewToAltAz(86.96717, 30.09397)
print('Slew complete')
mount.Tracking = False

print('Setting lights')
pulsar_dimmer.dimmer(254)
print('Lights set')

In [ ]:
save_loc = 'D:/Calibrations/2023-02-06 11_10_10/'
readoutLoop = [1]
binLoop = [1]
exposureLoop = [10]
filLoop = [6]
light = [1]; im_type = {0:'dark', 1:'flat'}
repeat = 25 # number of images per option
estimatedTime = len(readoutLoop)*len(binLoop)*len(filLoop)*len(light)*repeat*sum(exposureLoop)/(60*60)
print('Estimated time to complete: %.2f hours' % estimatedTime)

In [ ]:
for fil in filLoop:
    for shutter in light:
        for binning in binLoop:
            camera.BinX = binning
            for readout in readoutLoop:
                camera.ReadoutMode = readout
                for exp in exposureLoop:
                    for i in range(repeat):
                        while camera.Temperature > -19:
                            print('Waiting for temperature cooldown...')
                            time.sleep(10)
                        camera.Expose(exp, shutter, fil)
                        # savestring = save_loc + (im_type[shutter]+'_'+
                        #                 camera.ReadoutModes[readout].replace(' ', '')+'_'+
                        #                 str(binning)+'x'+str(binning)+'_'+
                        #                 camera.Filternames[fil]+'_'+
                        #                 str(exp).replace('.', '')+'s__'+
                        #                 str(i)+'.fts')
                        name = 'Flat_%03d%s.fits' % (i, camera.Filternames[fil])
                        savestring = save_loc + name
                        while not camera.ImageReady:
                            time.sleep(0.1)
                        print('Saving: %s' % savestring)
                        camera.SaveImage(savestring)

print('Turning off lights')
pulsar_dimmer.dimmer(0)
print('Lights off')

print('Homing')
mount.FindHome
mount.Tracking = False
print('Home found')